<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Stage-2---Increasing-granularity" data-toc-modified-id="Stage-2---Increasing-granularity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Stage 2 - Increasing granularity</a></span></li></ul></div>

The goal is to give a prediction of in which care unit will a patient be in in a certain number of days (or if they will be discharged). This way, hospital workload per care unit/department can be predicted. 

For this, I will have a table of patients with multiple features including medical data (diagnosis, medications...), previous stays and transfers, some demographic data, duration of stay... This can be done using the admissions table and joining information from the diagnosis table (specific per stay) 

At admission, which department will the patient end up in?
Will the patient visit this deparment in their stay?

Make more questions during the weekend.

Do the questions being clear about which variables you will use

And in two weeks have more or less the table ready. 

Before continuing, extract a smaller table that only contains relevant columns for finding atc1s. Find categories and fill them in that table. Then update/rejoin to prescriptions table and find transfer numbers.

I want to predict department occupancy in a hospital and I have a dataset that includes different information from patients. Initially at least, I am simplifying that by predicting instead the department that the patients are going to go to next. 

After a preliminary EDA I saw that I could not use the diagnosis data because it was not dated (it is a table with diagnosis for the whole stay), so now I am working to merge my database with a huge prescriptions table that is dated (so I can relate to patients for different stays and hospital transfers). I am converting the drugs to drug categories and did some frequency encoding, but there was like 20% of missing data, so I was filling that until recently. 

In principle I am working with 14 drug categories, but I have loads of data so could increase that to subcategories and apply PCA. In terms of modeling, my plan is to start with logistic regression and then move on to SVM, random forests...

In [21]:
import numpy as np
import pandas as pd
import copy
import nachopy
import importlib
import datetime

In [2]:
importlib.reload(nachopy);

In [3]:
prescriptions = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/prescriptions.csv')

C:\Users\Nacho\AppData\Local\Temp\ipykernel_19312\601855088.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/prescriptions.csv')


In [5]:
pres_dupl = prescriptions[['subject_id','hadm_id','starttime','stoptime','drug']].duplicated()

In [6]:
prescriptions = prescriptions[~pres_dupl]

In [7]:
medrecon = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/ed/medrecon.csv')

In [8]:
prescriptions['ndc'] = prescriptions.ndc.astype('Int64')
medrecon['ndc'] = medrecon.ndc.astype('Int64')

In [9]:
display(prescriptions.head())
display(prescriptions.info(show_counts = True))

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,11700683,10000032-34,34.0,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Acetaminophen,APAP500,004490,904198861,500mg Tablet,NaN,500,mg,1,TAB,NaN,PO/NG
1,10000032,22595853,14779570,10000032-22,22.0,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV
2,10000032,22595853,19796602,10000032-50,50.0,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,FURO40,008209,51079007320,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,20256254,10000032-32,32.0,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,RALT400,063231,6022761,400 mg Tablet,NaN,400,mg,1,TAB,2.0,PO
4,10000032,22595853,28781051,10000032-27,27.0,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Heparin,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,NaN,5000,UNIT,1,mL,3.0,SC


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14924364 entries, 0 to 16219411
Data columns (total 20 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   subject_id         14924364 non-null  int64  
 1   hadm_id            14924364 non-null  int64  
 2   pharmacy_id        14924364 non-null  int64  
 3   poe_id             14821476 non-null  object 
 4   poe_seq            14821476 non-null  float64
 5   starttime          14916988 non-null  object 
 6   stoptime           14911010 non-null  object 
 7   drug_type          14924364 non-null  object 
 8   drug               14924363 non-null  object 
 9   formulary_drug_cd  14904840 non-null  object 
 10  gsn                13294245 non-null  object 
 11  ndc                14898848 non-null  Int64  
 12  prod_strength      14917950 non-null  object 
 13  form_rx            20208 non-null     object 
 14  dose_val_rx        14917996 non-null  object 
 15  dose_unit_rx 

None

In [10]:
display(medrecon.head())
display(medrecon.info(show_counts = True))

,subject_id,stay_id,charttime,name,gsn,ndc,etc_rn,etccode,etcdescription
0,10000032,32952584,2180-07-22 17:26:00,albuterol sulfate,28090,21695042308,1,5970.0,Asthma/COPD Therapy - Beta 2-Adrenergic Agents...
1,10000032,32952584,2180-07-22 17:26:00,calcium carbonate,1340,10135021101,1,733.0,Minerals and Electrolytes - Calcium Replacement
2,10000032,32952584,2180-07-22 17:26:00,cholecalciferol (vitamin D3),65241,37205024678,1,670.0,Vitamins - D Derivatives
3,10000032,32952584,2180-07-22 17:26:00,emtricitabine-tenofovir [Truvada],57883,35356007003,1,5849.0,Antiretroviral - Nucleoside and Nucleotide Ana...
4,10000032,32952584,2180-07-22 17:26:00,fluticasone [Flovent HFA],21251,49999061401,1,371.0,Asthma Therapy - Inhaled Corticosteroids (Gluc...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143791 entries, 0 to 3143790
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   subject_id      3143791 non-null  int64  
 1   stay_id         3143791 non-null  int64  
 2   charttime       3143791 non-null  object 
 3   name            3143791 non-null  object 
 4   gsn             3143791 non-null  int64  
 5   ndc             3143791 non-null  Int64  
 6   etc_rn          3143791 non-null  int64  
 7   etccode         3131412 non-null  float64
 8   etcdescription  3131412 non-null  object 
dtypes: Int64(1), float64(1), int64(4), object(3)
memory usage: 218.9+ MB


None

Warning about ndc having mixed types. Instead of using isna to find about null values, we will just take as valid NDCs those that are larger that have at least 6 digits (which is the smallest NDC according to the FDA liink). We can also check whether we have other NDC numbers with fewer digits, as these could be fixable mistakes.

Prescriptions and medrecon are similar,thankfully, although instead of hadm_id we have stay_id. And no null values on our drugs column, yay! We lack the drug type column.

In [11]:
def digit_checker(df, d_min = 0, d_max = 6):
    for i in range(d_min,d_max):
        d_no = 10**i

        validndc_bool = df.ndc>=d_no
        print(f'% of NDCs >= 10^{i}:', round(validndc_bool/len(df)*100,2))
        print('Number of NDCs:', df.ndc[validndc_bool].nunique(),'\n')

In [12]:
# display(prescriptions.nunique())
nachopy.digit_checker(prescriptions,d_max = 12)

% of NDCs >= 10^0: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^1: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^2: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^3: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^4: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^5: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^6: 88.16
Number of NDCs: 5740 

% of NDCs >= 10^7: 83.9
Number of NDCs: 5468 

% of NDCs >= 10^8: 77.79
Number of NDCs: 4723 

% of NDCs >= 10^9: 32.43
Number of NDCs: 2965 

% of NDCs >= 10^10: 31.82
Number of NDCs: 2949 

% of NDCs >= 10^11: 0.0
Number of NDCs: 0 



In [13]:
display(medrecon.nunique())
digit_checker(medrecon,d_max = 12)

subject_id        148919
stay_id           323582
charttime         581453
name               19165
gsn                 9262
ndc                 9398
etc_rn                 5
etccode             1208
etcdescription      1212
dtype: int64

% of NDCs >= 10^0: 0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
3143786    0.0
3143787    0.0
3143788    0.0
3143789    0.0
3143790    0.0
Name: ndc, Length: 3143791, dtype: Float64
Number of NDCs: 9397 

% of NDCs >= 10^1: 0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
3143786    0.0
3143787    0.0
3143788    0.0
3143789    0.0
3143790    0.0
Name: ndc, Length: 3143791, dtype: Float64
Number of NDCs: 9397 

% of NDCs >= 10^2: 0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
3143786    0.0
3143787    0.0
3143788    0.0
3143789    0.0
3143790    0.0
Name: ndc, Length: 3143791, dtype: Float64
Number of NDCs: 9397 

% of NDCs >= 10^3: 0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
3143786    0.0
3143787    0.0
3143788    0.0
3143789    0.0
3143790    0.0
Name: ndc, Length: 3143791, dtype: Float64
Number of NDCs: 9397 



So all NDCs larger than 1 appear to be correct. There are 12.44% missing NDC, but we will not be filling these. We are interested on the ATR4 classification, so we will match the available NDCs to this class and then try to fill any empty values for that feature.

For the NDC matching we are using a function provided by tal that queries tal tal. The prescriptions table was used as input for this function and the output is uploaded and mapped into our table here.

In [14]:
prescriptions.drop(columns = ['prod_strength','form_rx','dose_val_rx','dose_unit_rx','form_val_disp',\
                              'form_unit_disp','doses_per_24_hrs','pharmacy_id','poe_id','poe_seq','stoptime'],inplace = True)


Dropping some of the columns as they (may) have no relevance as we are substituting the drug features by their categories. This will also allow us to combine our dfs.

In [15]:
print(sum(medrecon.duplicated()))
dupl_rows = medrecon[['stay_id','charttime','name']].duplicated()
print(sum(dupl_rows))

0
439227


Medications are duplicated to include the different etc codes for one drug. That is okay, we could create dummies and group the columns. However, we will not use the edc in principle. We are going to move to ATCs, which are more interpretable, general and well-recognised. More importantly, they have a very customisable granularity, as we can select the level. At level 0 we have only 14 features, which simplifies our model greatly. We will start with that and increase granularity if necessary.

In [16]:
medrecon.drop(columns =  ['etccode','etc_rn','etcdescription'], inplace = True)
medrecon = medrecon[~dupl_rows]
medrecon.rename(columns = {'charttime':'starttime','name':'drug'},inplace = True)

In [17]:
prescriptions

,subject_id,hadm_id,starttime,drug_type,drug,formulary_drug_cd,gsn,ndc,route
0,10000032,22595853,2180-05-07 01:00:00,MAIN,Acetaminophen,APAP500,004490,904198861,PO/NG
1,10000032,22595853,2180-05-07 00:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0,IV
2,10000032,22595853,2180-05-08 08:00:00,MAIN,Furosemide,FURO40,008209,51079007320,PO/NG
3,10000032,22595853,2180-05-07 01:00:00,MAIN,Raltegravir,RALT400,063231,6022761,PO
4,10000032,22595853,2180-05-07 00:00:00,MAIN,Heparin,HEPA5I,006549,63323026201,SC
...,...,...,...,...,...,...,...,...,...
16219407,19999987,23865745,2145-11-03 00:00:00,MAIN,Influenza Virus Vaccine,INFL0.5LF,069637,33332001201,IM
16219408,19999987,23865745,2145-11-04 10:00:00,MAIN,Levothyroxine Sodium,LEVO50,006649,74455211,PO/NG
16219409,19999987,23865745,2145-11-10 10:00:00,MAIN,Duloxetine,DULO30,057892,2324033,PO
16219410,19999987,23865745,2145-11-09 10:00:00,MAIN,Venlafaxine XR,VENL150XR,046405,68084048601,PO


In [19]:
nachopy.display_sbs([medrecon.stay_id.describe(),prescriptions.hadm_id.describe()],\
                    titles = ['Stay_id from medrecon','Hadm_id from prescriptions'])

,stay_id
count,2704564.000000
mean,35001695.562961
std,2885678.657628
min,30000012.000000
25%,32502217.000000
50%,35000352.000000
75%,37497977.000000
max,39999964.000000
,hadm_id
count,14924364.000000


Lovely, comparable, no overlap. We will combine them, adding a new column to indicate rows where hadm_id is actually stay_id (for info not for modelling).

In [22]:
prescriptions = pd.concat([prescriptions,medrecon.rename(columns = {'stay_id':'hadm_id'})],axis = 0)
prescriptions.reset_index(inplace = True,drop = True)

In [35]:
prescriptions

,subject_id,hadm_id,starttime,drug_type,drug,formulary_drug_cd,gsn,ndc,route
0,10000032,22595853,2180-05-07 01:00:00,MAIN,Acetaminophen,APAP500,004490,904198861,PO/NG
1,10000032,22595853,2180-05-07 00:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0,IV
2,10000032,22595853,2180-05-08 08:00:00,MAIN,Furosemide,FURO40,008209,51079007320,PO/NG
3,10000032,22595853,2180-05-07 01:00:00,MAIN,Raltegravir,RALT400,063231,6022761,PO
4,10000032,22595853,2180-05-07 00:00:00,MAIN,Heparin,HEPA5I,006549,63323026201,SC
...,...,...,...,...,...,...,...,...,...
17628923,19999828,32917002,2149-01-08 09:41:00,NaN,fluticasone,NaN,18368,12280027016,NaN
17628924,19999828,32917002,2149-01-08 09:41:00,NaN,glipizide,NaN,1776,11845110101,NaN
17628925,19999828,32917002,2149-01-08 09:41:00,NaN,ibuprofen,NaN,8350,10544007120,NaN
17628926,19999828,32917002,2149-01-08 09:41:00,NaN,metronidazole,NaN,9592,10544093814,NaN


In [36]:
print(sum(prescriptions.duplicated()))
dupl_rows = prescriptions[['subject_id','hadm_id','starttime','drug','ndc']].duplicated()
print(sum(dupl_rows))

337075
377114


In [ ]:
prescriptions['starttime'] = pd.to_datetime(prescriptions.starttime)

In [ ]:
nachopy.display_sbs([prescriptions.ndc[prescriptions.stay_id_ind == 1].describe(),\
                      prescriptions.ndc[prescriptions.stay_id_ind == 0].describe()],\
                      titles = ['NDCs from hospital','NDCs from ED'])

It seems like ED and the hospital have completely different suppliers, because there is little overlap of ndcs. Let's export the unique ndc list to query the ATCs.

In [24]:
mapped_NDC = pd.read_csv('G:/My Drive/Capstone-Data/ndc_map/output/atc4/ndc_map 2022_10_12 17_12 (atc4).csv',usecols = ['ndc','atc4','atc4_name'])

In [25]:
mapped_NDC.dropna(inplace = True)
mapped_NDC.ndc = mapped_NDC.ndc.astype('int64')

In [26]:
mapped_NDC.drop_duplicates(inplace = True)
len(mapped_NDC)

21710

In [27]:
mapped_NDC.head()

,ndc,atc4,atc4_name
2,2050301,J01GB,Other aminoglycosides
3,2050301,S01AA,Antibiotics
4,2060440,J04AB,Antibiotics
5,2100902,R05DA,Opium alkaloids and derivatives
8,2143301,A10BJ,Glucagon-like peptide-1 (GLP-1) analogues


In [28]:
mapped_NDC.atc4_name.nunique()

518

In [32]:
mapped_NDC['atc3'] = mapped_NDC.atc4.str[:-1]
mapped_NDC['atc2'] = mapped_NDC.atc4.str[:3]
mapped_NDC['atc1'] = mapped_NDC.atc4.str[0]

print('Number of categories')
print(f'ATC4: {mapped_NDC.atc4.nunique()}\nATC3: {mapped_NDC.atc3.nunique()}\nATC2: {mapped_NDC.atc2.nunique()}\nATC1: {mapped_NDC.atc1.nunique()}')

Number of categories
ATC4: 560
ATC3: 196
ATC2: 86
ATC1: 14


A lot of categories. Feasible, but we will first try taking just the first letter (lvl 1 of classification) of the atc_4 instead

In [62]:
atc1 = mapped_NDC[['ndc','atc4']].copy()
atc1['atc4'] = atc1.atc4.str[0]
atc1.rename(columns = {'atc4':'atc1'},inplace = True)
atc1.drop_duplicates(inplace = True)

In [64]:
atc1 = pd.concat([mapped_NDC.atc1,pd.get_dummies(atc1.atc1,prefix = None)],axis = 1).drop(columns = 'atc1')
atc1 = atc1.groupby('ndc').any()*1
display(atc1.head())
print(atc1.shape)

,A,B,C,D,G,H,J,L,M,N,P,R,S,V
ndc,,,,,,,,,,,,,,
2050301,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2060440,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2100902,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2143301,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2143401,1,0,0,0,0,0,0,0,0,0,0,0,0,0


(10770, 14)


There is more than one atc1 per ndc code! We were going to use dummy variables in the future, so we will implement those now before the merge. That way we will have just one category per ndc. Also note that not all ndcs have been mapped, 490 out of 5744 are still unclassified. We will try to fill those after the merge.

In [65]:
prescriptions.reset_index(inplace = True)
prescriptions.rename(columns = {'index':'join_on_index'},inplace = True)
prescriptions.set_index('ndc',inplace = True)
cat_pres = prescriptions[['join_on_index','drug_type','drug','formulary_drug_cd','gsn']].join(atc1)
cat_pres.reset_index(inplace = True)

In [66]:
display(cat_pres.sample(10))
print('Missing categories %:',sum(cat_pres.A.isna())/len(cat_pres)*100)
cat = list('ABCDGHJLMNPRSV')
cat_pres[cat] = cat_pres[cat].fillna(-1).astype('int8')

print(cat_pres.info(show_counts = True))

,ndc,join_on_index,drug_type,drug,formulary_drug_cd,gsn,A,B,C,D,G,H,J,L,M,N,P,R,S,V
9308855,904516561.0,2504329,MAIN,Senna,SENN187,019964,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14699936,51079096620.0,6085387,MAIN,Famotidine,PEPC20,011677,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7168339,409665305.0,3543575,MAIN,Potassium Chloride,KCL40I,001255,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
418017,0.0,3574543,BASE,SW,KCLBASE2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5783571,338070948.0,14351113,MAIN,Potassium Chloride,KCL10PM,015362,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11185323,10544034130.0,17151427,NaN,omeprazole,NaN,33530,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13747385,50419040201.0,15069592,NaN,drospirenone-ethinyl estradiol [Ocella],NaN,47787,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
14662615,51079093020.0,3916219,MAIN,Carvedilol,CARV625,028109,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5742074,338070341.0,6630053,MAIN,Potassium Chloride,KCL20PM,045309,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16270188,63323026965.0,12987648,MAIN,Propofol,PROP100IG,016796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Missing categories %: 17.853161576245586
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628928 entries, 0 to 17628927
Data columns (total 20 columns):
 #   Column             Non-Null Count     Dtype 
---  ------             --------------     ----- 
 0   ndc                17603412 non-null  object
 1   join_on_index      17628928 non-null  int64 
 2   drug_type          14924364 non-null  object
 3   drug               17628927 non-null  object
 4   formulary_drug_cd  14904840 non-null  object
 5   gsn                15998809 non-null  object
 6   A                  17628928 non-null  int8  
 7   B                  17628928 non-null  int8  
 8   C                  17628928 non-null  int8  
 9   D                  17628928 non-null  int8  
 10  G                  17628928 non-null  int8  
 11  H                  17628928 non-null  int8  
 12  J                  17628928 non-null  int8  
 13  L                  17628928 non-null  int8  
 14  M                  17628928 non-null  i

In [67]:
cat_pres['ndc'] = cat_pres.ndc.floordiv(100)
cat_pres.loc[cat_pres.ndc == 0,'ndc'] = pd.NA
cat_pres['ndc'] = cat_pres.ndc.astype('Int64')

Note that the new category dummy columns are floats after the join, not int8s. We have filled the NaNs as -1 and changed the data type of the columns. This is useful to decrease the size of the dataframe and it will also be useful to fill the missing values (which is 19.26% of the rows), as we will see below. We will fill these missing values by matching with other columns and we there are a few candidates column for this: drug, pharmacy_id, gsn and formulary_drug_cd. The goal will be to find the column with values shared by the largest amount of rows, so matching ATR1s is more likely. 

In [68]:
nndrug = cat_pres[cat_pres.drug.isna()].index.values[0]
display(cat_pres.loc[nndrug])

display(cat_pres[cat_pres.ndc == cat_pres.ndc.loc[nndrug]].head(1))

cat_pres.loc[nndrug,['drug', 'formulary_drug_cd']] = ('Simvastatin','SIMV40')

print('Value counts of drugs:\n',cat_pres.drug.value_counts())
cat_pres.drug = cat_pres.drug.str.lower()
drugs_vc = cat_pres.drug.value_counts()

clin_drug = cat_pres.drug.str.contains('clindamyci')
display(cat_pres[clin_drug].head(1))
cat_pres.drug[clin_drug] = 'clindamycin'

print('\n\nValue counts of drugs after making all letters lowercase:\n',drugs_vc)
print('\nDescription:\n',drugs_vc.describe())

print('\nNumber of drugs used only once:', sum(drugs_vc>1))

cat_pres['drug'] = cat_pres.drug.str.replace('[^a-zA-Z]','', regex = True)
drugs_vc = cat_pres.drug.value_counts()

print('\n\nValue counts of drugs after making all letters lowercase:\n',drugs_vc)
print('\nDescription:\n',drugs_vc.describe())

print('\nNumber of drugs used only once:', sum(drugs_vc>1))



ndc                  510790456
join_on_index         14845444
drug_type                 MAIN
drug                       NaN
formulary_drug_cd      SIMVIND
gsn                     016579
A                            0
B                            0
C                            1
D                            0
G                            0
H                            0
J                            0
L                            0
M                            0
N                            0
P                            0
R                            0
S                            0
V                            0
Name: 14378694, dtype: object

,ndc,join_on_index,drug_type,drug,formulary_drug_cd,gsn,A,B,C,D,G,H,J,L,M,N,P,R,S,V
14358871,510790456,333,MAIN,Simvastatin,SIMV40,016579,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Value counts of drugs:
 Insulin                        557829
Sodium Chloride 0.9%  Flush    530628
0.9% Sodium Chloride           508792
Acetaminophen                  411170
Furosemide                     333397
                                ...  
*ramapril                           1
*adderral                           1
*ellipta                            1
*bisa-lax                           1
Lumigan                             1
Name: drug, Length: 25853, dtype: int64


,ndc,join_on_index,drug_type,drug,formulary_drug_cd,gsn,A,B,C,D,G,H,J,L,M,N,P,R,S,V
1742431,<NA>,15092293,NaN,*clindamycin 1%,NaN,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


C:\Users\Nacho\AppData\Local\Temp\ipykernel_1612\2913060166.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_pres.drug[clin_drug] = 'clindamycin'




Value counts of drugs after making all letters lowercase:
 insulin                                          557853
sodium chloride 0.9%  flush                      530628
0.9% sodium chloride                             508792
acetaminophen                                    441995
furosemide                                       359143
                                                  ...  
desog-e.estradiol/e.estradiol [pimtrea (28)]          1
desog-e.estradiol/e.estradiol [kimidess (28)]         1
*genov                                                1
*steralive                                            1
lumigan                                               1
Name: drug, Length: 22976, dtype: int64

Description:
 count     22976.000000
mean        767.275766
std       10722.416656
min           1.000000
25%           1.000000
50%           3.000000
75%          17.000000
max      557853.000000
Name: drug, dtype: float64

Number of drugs used only once: 13943


Value counts of 

In [69]:
empty_drug = cat_pres.drug ==u''
print(f'There are {sum(empty_drug)} empty drugs')
display(cat_pres[empty_drug].sample())

fdrugs_empty = cat_pres.formulary_drug_cd[empty_drug].drop_duplicates().values
empty_drug = cat_pres.formulary_drug_cd.isin(fdrugs_empty)

cat_pres.drug.where(~empty_drug,cat_pres.formulary_drug_cd,inplace = True)

print(f'{sum(drugs_vc == 1)/len(cat_pres)*100}% of values in drugs column appears only once')

There are 112 empty drugs


,ndc,join_on_index,drug_type,drug,formulary_drug_cd,gsn,A,B,C,D,G,H,J,L,M,N,P,R,S,V
1743711,<NA>,15286317,NaN,,NaN,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


0.04262879739482741% of values in drugs column appears only once


There was just one missing value for the drug column, which was easily filled by checking elsewhere in the table. After this, we analysed the number of rows for each drug. Interestingly, the formulary_drug_code was different than the original for that row. 

The input of these fields does not seem consistent. For example "Clindamycin" is misspelt (fixed for this particular case), abbreviations may not be consistent and upper and lower case seem to be used with little care. This last issue could be fixed with str.lower(), though. 

This means too many groups so matching categories will be harder. Let us look at the other columns with drug identifiers.

In [70]:
print('gsn value counts:\n',cat_pres.gsn.value_counts())

fd_vc = cat_pres.formulary_drug_cd.value_counts()
print('\n\nformulary_drug_cd value counts:\n',fd_vc)
print('\nDescription:\n',fd_vc.describe())

fd_vc_5 = fd_vc[fd_vc<=5].value_counts()
print('\nNumber of formulary_drug_cd present on only 5 rows or less:\n',fd_vc_5)
print('\n% of rows with drugs used 5 times or less:', round(fd_vc_5.sum()/fd_vc.sum()*100,5))

gsn value counts:
 001210                          757416
006549                          320343
027413                          252990
008205                          247213
004489                          242690
                                 ...  
003746                               1
002485                               1
045006 064197                        1
8245                                 1
021723 021726 021725 021724          1
Name: gsn, Length: 15601, dtype: int64


formulary_drug_cd value counts:
 NACLFLUSH       530435
HEPA5I          320637
ACET325         243655
DOCU100         243283
SENN187         206732
                 ...  
VANC.000125L         1
VANCOD125L           1
VANC1.25L            1
LEVO2                1
SAGE547/PCB          1
Name: formulary_drug_cd, Length: 3789, dtype: int64

Description:
 count      3789.000000
mean       3933.713381
std       18224.888227
min           1.000000
25%          10.000000
50%          97.000000
75%        1118.0000

From these candidates, formulary_drug_cd, gsn, and drugs seems the best to find the right ATR1s, but they have many null values. Additionally, the gsn column has 1.8 M missing values and sometimes several codes in one field. We will have to work around this
The pharmacy_id has many unique values, so this variable is not specific to the same drug, just an inventory code specific per item.  

We want to decrease the number of unique values as this will increase our chances of filling missing ATR1 fields. All our columns have many null values, but we can recursively fill categories based on all the identifying columns.

In [71]:
len(cat_pres)

17628928

In [72]:
cat_pres[cat].sum(axis = 1).value_counts()

 1     9481544
-14    3147321
 2     2660944
 3     1582873
 4      350029
 5      297370
 6       87202
 7       21059
 8         586
dtype: int64

In [73]:
cat_pres.gsn.value_counts()

001210                          757416
006549                          320343
027413                          252990
008205                          247213
004489                          242690
                                 ...  
003746                               1
002485                               1
045006 064197                        1
8245                                 1
021723 021726 021725 021724          1
Name: gsn, Length: 15601, dtype: int64

In [74]:
nachopy.rec_cat_filler(cat_pres,['gsn','drug','formulary_drug_cd','ndc'])

% of missing values before loop: 17.853161576245586
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 16.9409058505082
Loop #1, missing values % after filling with drug column: 10.549619660034407
Loop #1, missing values % after filling with formulary_drug_cd column: 9.868023200005386
Loop #1, missing values % after filling with ndc column: 9.663048752943428

Loop #1, missing values % after re-imploding df: 9.634953412935829
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 9.611727364357368
Loop #2, missing values % after filling with drug column: 9.002189007403166
Loop #2, missing values % after filling with formulary_drug_cd column: 8.987510400175035
Loop #2, missing values % after filling with ndc column: 8.979721405125657

Loop #2, missing values % after re-imploding df: 8.9822875219639
Dataframe exploded, starting for loop #3

Loop #3, missing values

In [75]:
importlib.reload(nachopy)

<module 'nachopy' from 'C:\\Users\\Nacho\\Capstone-project-Brainstation\\nachopy.py'>

In [76]:
nachopy.display_sbs([cat_pres.drug[cat_pres.A<0].value_counts(normalize = True).iloc[0:50]*100],\
                    titles = ['Drugs value counts'],max_rows = 25)


,drug
isoosmoticdextrose,20.622194
bag,15.210765
vial,7.228978
polyethyleneglycol,7.047468
influenzavaccinequadrivalent,5.838108
neutraphos,5.117210
influenzavirusvaccine,5.024741
simethicone,3.390417
soln,3.070489
pneumococcalvacpolyvalent,2.874534


Lots of vaccines, let us fill those first

In [77]:
cat = list('ABCDGHJLMNPRSV')
vaccines_bool = (cat_pres.drug.str.contains('vaccine|polyvalent|measlesmumpsrubellavac',regex = True, na = False))
vaccines_bool *= cat_pres.A<0
cat_pres.loc[vaccines_bool,cat] = 0
cat_pres.loc[vaccines_bool,'J'] = 1

print('% of missing values:',nachopy.cat_check(cat_pres)*100)

% of missing values: 7.856632008480606


Let us have a look at the missing values and the drugs per drug type (from the prescriptions table)

In [78]:
print('% of missing values for MAIN drugs',nachopy.cat_check(cat_pres[cat_pres.drug_type =='MAIN'])*100)
print('% of missing values for BASE drugs',nachopy.cat_check(cat_pres[cat_pres.drug_type =='BASE'])*100)
print('% of missing values for ADDITIVE drugs',nachopy.cat_check(cat_pres[cat_pres.drug_type =='ADDITIVE'])*100)

base_drugs_bool = cat_pres.drug_type == 'BASE'
print('\n',cat_pres.drug[base_drugs_bool].drop_duplicates().values)

% of missing values for MAIN drugs 3.185890591466149
% of missing values for BASE drugs 25.785027717319707
% of missing values for ADDITIVE drugs 4.945122703169318

 ['sodiumchloride' 'isoosmoticdextrose' 'sw' 'dextrose' 'bag' 'vial'
 'isoosmoticsodiumchloride' 'ns' 'isotonicsodiumchloride'
 'syringesodiumchloride' 'soln' 'dw' 'syringe' 'sterilewater'
 'syringechemo' 'mcgvial' 'yellowcaddcassette' 'syringeisoosmoticdextrose'
 'epiduralbag' 'amp' 'bluecaddcassette' 'nssyringeintrapleural'
 'syringesterilewater' 'syringelr' 'syringesw' 'nsepiduralbag'
 'clickchangetochooseabasesolution' 'sodiumchlorideexcelbag' 'nssyringe'
 'prismasatebk' 'sodiumchlyellowcadd' 'syringesns' 'bottle' 'syringens'
 'solution' 'sodiumchloridepfsyringe' 'aerogensyringens' 'syringepca'
 'syringeintraperitoneal' 'syringepd' 'dwglassbottle' 'aerogensyringesw'
 'dwwalbumin' 'tbsyringe' 'viaflexbag' 'sodiumcitrate'
 'syringeintraventricular' 'caddcassette' 'syringeinhalation'
 'syringeintrathecal' 'nsintrathecalbag

Rows with drug_type "BASE" correspond with procedures (bag, vial...), solvents (sodium chloride, dextrose) and treatments/drugs (lipid/fat emulsions, travasol, amino acids...) apparently using IV therapy. Many of these procedures will not be very specific per hospital department (sodium chloride, bags, dextrose...), but frequency of procedures could be interesting so registering these events in our table could be useful if we use frequency encoding instead of one-hot encoding. Given that most of these events involve IV therapy and solvents are classed typically both as B (Blood products) and as V (Various) in the ATC system, these prescription events have been classified as such. 

In [79]:
base_drugs_bool = base_drugs_bool * cat_pres.A<0 
cat_pres.loc[base_drugs_bool,cat] = 0

cat_pres.loc[base_drugs_bool,'V'] = 1
cat_pres.loc[base_drugs_bool,'B'] = 1
cat_pres.A.value_counts(normalize = True)*100

print('% of missing values:',nachopy.cat_check(cat_pres)*100)

% of missing values: 4.237109596227292


We still have 4.03% of missing values in our atc1 columns. Let us run the recursive filler function again.


In [80]:
nachopy.rec_cat_filler(cat_pres,['gsn','drug','formulary_drug_cd','ndc'])

% of missing values before loop: 4.237109596227292
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 4.230159656297725
Loop #1, missing values % after filling with drug column: 4.1881714861804165
Loop #1, missing values % after filling with formulary_drug_cd column: 4.1881714861804165
Loop #1, missing values % after filling with ndc column: 4.18801310429125

Loop #1, missing values % after re-imploding df: 4.179573482857267
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 4.18801310429125
Loop #2, missing values % after filling with drug column: 4.18801310429125
Loop #2, missing values % after filling with formulary_drug_cd column: 4.18801310429125
Loop #2, missing values % after filling with ndc column: 4.18801310429125

Loop #2, missing values % after re-imploding df: 4.179573482857267


We can try and fill manually the drugs that occur most frequently.We can do another round of recursive filling

https://go.drugbank.com/drugs/DB09512

In [81]:
missing_drugs = cat_pres.drug[cat_pres.A<0].value_counts(normalize = True).head(30)
display(missing_drugs*100)
print(f'We will fill {round(missing_drugs.sum()*100,2)}% of the remaining missing drugs manually')
missing_drugsl = missing_drugs.index.values

polyethyleneglycol                      22.796416
neutraphos                              16.552620
simethicone                             10.966969
potassiumphosphate                       6.884517
phosphorus                               5.459197
sarnalotion                              4.291537
epoetinalfa                              4.125747
readicatbariumsulfatesuspension          3.894750
tiotropiumbromide                        3.433283
sodiumpolystyrenesulfonate               2.693563
vasopressin                              2.027762
ferricgluconate                          1.531711
tuberculinprotein                        1.226267
leucovorincalcium                        1.095588
gastroviewdiatrizoatemegluminesodium     1.058100
lithiumcarbonate                         0.842943
psylliumwafer                            0.766912
gammagardliquid                          0.764800
caphosol                                 0.606401
vitaminbcomplex                          0.601913


We will fill 95.78% of the remaining missing drugs manually


In [82]:
cat_pres.loc[cat_pres.drug.isin(missing_drugsl),cat] = 0

cat_pres.loc[cat_pres.drug == 'polyethyleneglycol','B'] = 1
cat_pres.loc[cat_pres.drug == 'neutraphos',['A','B']] = 1
cat_pres.loc[cat_pres.drug == 'simethicone','A'] = 1
cat_pres.loc[cat_pres.drug == 'potassiumphosphate','B'] = 1
cat_pres.loc[cat_pres.drug == 'phosphorus','B'] = 1
cat_pres.loc[cat_pres.drug == 'epoetinalfa','B'] = 1
cat_pres.loc[cat_pres.drug == 'sarnalotion', 'D'] = 1
cat_pres.loc[cat_pres.drug == 'readicatbariumsulfatesuspension',['A','V']] = 1 # Contrast agent
cat_pres.loc[cat_pres.drug == 'tiotropiumbromide','R'] = 1
cat_pres.loc[cat_pres.drug == 'sodiumpolystyrenesulfonate','V'] = 1
cat_pres.loc[cat_pres.drug == 'vasopressin','H'] = 1
cat_pres.loc[cat_pres.drug == 'fishoilmega','C'] = 1
cat_pres.loc[cat_pres.drug == 'ferricgluconate','B'] = 1
cat_pres.loc[cat_pres.drug == 'leucovorincalcium','V'] = 1
cat_pres.loc[cat_pres.drug == 'tuberculinprotein','V'] = 1
cat_pres.loc[cat_pres.drug == 'gastroviewdiatrizoatemegluminesodium',['A','V']] = 1 # Contrast agent
cat_pres.loc[cat_pres.drug == 'lithiumcarbonate','N'] = 1
cat_pres.loc[cat_pres.drug == 'gammagardliquid', 'J'] = 1
cat_pres.loc[cat_pres.drug == 'psylliumwafer', 'A'] = 1
cat_pres.loc[cat_pres.drug == 'caphosol', 'V'] = 1
cat_pres.loc[cat_pres.drug == 'vitaminbcomplex', 'B'] = 1
cat_pres.loc[cat_pres.drug == 'hydrocerin','D'] = 1
cat_pres.loc[cat_pres.drug == 'fosaprepitant','A'] = 1
cat_pres.loc[cat_pres.drug == 'psyllium', 'A'] = 1
cat_pres.loc[cat_pres.drug == 'levalbuterolneb','R'] = 1
cat_pres.loc[cat_pres.drug == 'aquaphorointment','D'] = 1
cat_pres.loc[cat_pres.drug == 'psylliumpowder','A'] = 1
cat_pres.loc[cat_pres.drug == 'protaminesulfate','V'] = 1
cat_pres.loc[cat_pres.drug == 'transplantselfmedicationprogram','V'] = 1
cat_pres.loc[cat_pres.drug == 'bismuthsubsalicylate','A'] = 1
cat_pres.loc[cat_pres.drug == 'xopenexneb','R'] = 1

In [83]:
nachopy.rec_cat_filler(cat_pres,['gsn','drug','formulary_drug_cd','ndc'])

% of missing values before loop: 2.1215980915005153
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 2.112882279436674
Loop #1, missing values % after filling with drug column: 2.112650363098966
Loop #1, missing values % after filling with formulary_drug_cd column: 2.111875423141257
Loop #1, missing values % after filling with ndc column: 2.103503808999587

Loop #1, missing values % after re-imploding df: 2.089236509446292
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 2.1017955471950027
Loop #2, missing values % after filling with drug column: 2.1017842342029196
Loop #2, missing values % after filling with formulary_drug_cd column: 2.1017842342029196
Loop #2, missing values % after filling with ndc column: 2.1017842342029196

Loop #2, missing values % after re-imploding df: 2.087733298360513
Dataframe exploded, starting for loop #3

Loop #3, missing 

In [84]:
catnas = cat_pres[cat_pres.A < 0]
catnas[['ndc','drug_type','drug','formulary_drug_cd','gsn']].isna().sum()/len(catnas)*100

ndc                   7.399508
drug_type            94.689407
drug                 96.884561
formulary_drug_cd    96.884561
gsn                  95.162184
dtype: float64

In [85]:
cat_pres.ndc[cat_pres.A<0].value_counts(normalize = True).iloc[50]*100

0.3180704508443245

In [86]:
nachopy.display_sbs([cat_pres.ndc[cat_pres.A<0].value_counts(normalize = True).iloc[:60]*100],\
                    titles = ['ndc value counts'], max_rows = 10)
            

missing_ndcs = cat_pres.ndc[cat_pres.A<0].value_counts(normalize = True).head(30)
print(f'We could fill {round(missing_ndcs.sum()*100,2)}% of the remaining missing ndcs manually by filling the first 30 ndcs')

,ndc
100030116,7.928287
105850865,7.021317
100030115,6.088819
105720810,4.999633
107700128,4.999340
353560215,2.331245
109390331,1.748507
109390803,1.675151
123330025,1.600035
109390021,1.284019


We could fill 54.06% of the remaining missing ndcs manually by filling the first 30 ndcs


The problem is that the missing values are more distributed between many ndcs, so filling manually 30 of them would just tackle 53.83% of the missing categories by ndc. Moreover, browsing ndc is not so easy, as they are specific per supplier and can be harder to find. Even more if they could not be found on the FDA website. We will leave these values unfilled for now. We can save the merged, filled prescriptions df to be used later.

In [88]:
cat.append('join_on_index')
cat_pres = cat_pres[cat].set_index('join_on_index',drop = True)
prescriptions = prescriptions[['subject_id','hadm_id','starttime','join_on_index']].set_index('join_on_index', drop = True)
prescriptions_final = prescriptions.join(cat_pres)

In [90]:
prescriptions_final.to_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_final.pkl')

## Stage 2 - Increasing granularity 

Let's increase the granularity of the prescriptions. 

In [33]:
mapped_NDC.nunique()

ndc          10770
atc4           560
atc4_name      518
atc3           196
atc2            86
atc1            14
dtype: int64

ATC2 refers to the therapeutic subgroup, atc3 is specific to a pharmacological subgroup and atc4 is the chemical. We can do atc2, let's repeat the process above for this category. 

In [40]:
cat = list('ABCDGHJLMNPRSV')

In [42]:
prescriptions_v2

,subject_id,hadm_id,starttime,A,B,C,D,G,H,J,L,M,N,P,R,S,V
join_on_index,,,,,,,,,,,,,,,,,
0,10000032,22595853,2180-05-07 01:00:00,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,10000032,22595853,2180-05-07 00:00:00,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,10000032,22595853,2180-05-08 08:00:00,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,10000032,22595853,2180-05-07 01:00:00,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,10000032,22595853,2180-05-07 00:00:00,0,1,1,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17628923,19999828,32917002,2149-01-08 09:41:00,0,0,0,1,0,0,0,0,0,0,0,1,0,0
17628924,19999828,32917002,2149-01-08 09:41:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0
17628925,19999828,32917002,2149-01-08 09:41:00,0,0,1,0,1,0,0,0,1,0,0,1,0,0


In [38]:
prescriptions_v2 = pd.read_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_final.pkl')
prescriptions_v2 = prescriptions.drop(columns = cat).join()

In [ ]:
atc1 = pd.concat([mapped_NDC.atc1,pd.get_dummies(atc1.atc1,prefix = None)],axis = 1).drop(columns = 'atc1')
atc1 = atc1.groupby('ndc').any()*1
display(atc1.head())
print(atc1.shape)